In [ ]:
#importing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import smart_open

import logging
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix


import multiprocessing
import docx2txt

from gensim.models import Word2Vec
from sklearn.manifold import TSNE

import texthero as hero

In [ ]:
# !pip install gensim==3.4.0
# !pip install smart_open==1.9.0

In [ ]:
df = pd.read_csv("C:/Users/valli/Google Drive/Dell/Semester 6/Capstone/Data/SFW_13_cleaned_summarised.csv")

In [ ]:
df2 = pd.read_csv("C:/Users/valli/Google Drive/Dell/Semester 6/Capstone/Data/SFW_16_cleaned_summarised.csv")

In [ ]:
sf = pd.concat([df, df2], sort = False, ignore_index = True).drop(columns = ['Unnamed: 0'])
sf

In [ ]:
sf[(sf['Skill Title']=="Accident and Incident Response Management")&(sf['Sector']=="Air Transport")].sort_values('Proficiency Level')

In [ ]:
sf[(sf['Skill Title']=="Accident and Incident Response Management")&(sf['Sector']=="Tourism")].sort_values('Proficiency Level')

In [ ]:
skills_summarised = sf[['Sector', 'Skill Title', 'Proficiency Level', 'summary_jd']].groupby(['Sector', 'Skill Title', 'Proficiency Level']).describe()

In [ ]:
skills_summarised

In [ ]:
sectors = sf['Sector'].value_counts()

In [ ]:
sf_new = sf[~sf['Sector'].isin(sectors[sectors == 1].index)]

In [ ]:
sf_model = sf_new[["Job Role","clean_docs", "summary_jd", "Sector", "Skill Title", "Proficiency Level"]]

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from Portal import *
# pd.set_option('display.width', 400)
# pd.set_option('display.max_columns', 10)


class CareersFuture(Portal):

    def __init__(self, name):
        super().__init__(name)

    def read_data(self):
        # Reads data
        df = pd.read_csv('mycareerfuture2020-09-15.csv', error_bad_lines=False)
        return df

    def clean_data(self):
        df = self.read_data()
        # df = df[df['Job Description'].str.contains('value')]
        df['Job Description'] = df['Job Description'].astype(str)
        df['Job Description'] = df['Job Description'].apply(lambda x: self.strip_html_tags(x))

        df['Industry'] = df['Industry'].astype(str)
        df['Industry'] = df['Industry'].apply(lambda x: self.str_to_literal(x))

        # df['Skills'] = df['Skills'].astype(str)

        df['Job Experience Required (years)'] = df['Job Experience Required (years)'].astype(int)

        df['Job Monthly Min Sal'] = df['Job Monthly Min Sal'].astype(int)
        df['Job Monthly Max Sal'] = df['Job Monthly Max Sal'].astype(int)

        return df

    def str_to_literal(self, text):
        try:
            # ast.literal_eval function converts takes in a string and converts it into a dictionary
            ls = super().str_to_literal(text)
            industry = []
            for i in ls:
                industry.append(i['category'])
            return industry
        except:
            pass

careersfuture = CareersFuture("careersfuture")


In [ ]:
cf = pd.DataFrame(careersfuture.clean_data())

In [ ]:
cf['Industry_unlisted'] = cf['Industry'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
cf['Industry_unlisted'].unique()

In [ ]:
cf['Skills'] = cf['Skills'].apply(lambda x: x.replace("'", "") )

In [ ]:
cf['Skills_unlisted'] = cf['Skills'].apply(lambda x: x.replace('[','').replace(']',''))


In [ ]:
cf_model = cf[["Job Title", "Job Description", "Industry_unlisted", "Skills_unlisted", "Job Experience Required (years)","Job Monthly Min Sal", "Job Monthly Max Sal" ]]

In [ ]:
industries = pd.DataFrame(cf_model.Industry_unlisted.value_counts().nlargest(50))

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """

    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    

# print_plot(10)

In [ ]:
sf_model['clean_docs'] = sf_model['clean_docs'].astype(str).apply(clean_text)

In [ ]:
sf_model['summary_jd'] = sf_model['summary_jd'].astype(str).apply(clean_text)

In [ ]:
sf_model['Proficiency Level'] = sf_model['Proficiency Level'].str.split(n=1).str[1]

In [ ]:
sf_model

In [ ]:
sf_skills_concatenated = sf_model.groupby(['Sector','Skill Title','Proficiency Level'])['clean_docs'].unique()
sf_skills_concatenated = pd.DataFrame(sf_skills_concatenated)

In [ ]:
sf_skills_concatenated  = sf_skills_concatenated.groupby(['Sector','Skill Title'])['clean_docs']
sf_skills_concatenated = pd.DataFrame(sf_skills_concatenated )

In [ ]:
def concat_values(df):
    jd_list = []
    counter = 0
    for row in df[1]:
        jd=""
        counter+=1
        for val in row:
            jd+=val[0]
        jd_list.append(jd)
    return jd_list
        


In [ ]:
jd_list = concat_values(sf_skills_concatenated)

In [ ]:
sf_skills_concatenated['Skill description'] = jd_list
sf_skills_concatenated[['Sector', 'Skill Title']] = pd.DataFrame(sf_skills_concatenated[0].tolist()) 
sf_skills_concatenated = sf_skills_concatenated.drop(columns = [0, 1])
sf_skills_concatenated = sf_skills_concatenated[['Sector', 'Skill Title', 'Skill description']]

In [ ]:
sf_model

In [ ]:
sf_model_new = sf_model.merge(sf_skills_concatenated, on=["Sector","Skill Title"], how = 'left')

In [ ]:
sf_model_final = sf_model_new[['Job Role', 'Sector', 'Skill Title', 'Skill description']]

In [ ]:
sf_model_final

In [ ]:
sf_model_final.to_csv(index = False, path_or_buf="sf_skills_concat.csv")

In [ ]:
sf_model_final = pd.read_csv("sf_skills_concat.csv")

### Model

In [ ]:
sf_model_final['Skill description'] = hero.remove_stopwords(sf_model_final['Skill description'].astype(str))
sf_model_final['Skill description'] = hero.remove_urls(sf_model_final['Skill description'])
sf_model_final['Skill description'] = hero.tokenize(sf_model_final['Skill description']).astype(str)
sf_model_final['Skill description'] = hero.clean(sf_model_final['Skill description'])

In [ ]:
cf_model['Job Description'] = hero.remove_stopwords(cf_model['Job Description'].astype(str))
cf_model['Job Description'] = hero.remove_urls(cf_model['Job Description'])
cf_model['Job Description'] = hero.tokenize(cf_model['Job Description']).astype(str)
cf_model['Job Description'] = hero.clean(cf_model['Job Description'])

In [ ]:
from gensim.models.phrases import Phrases, Phraser
jobD = [row.split() for row in sf_model_final.loc[:, 'Skill description']]
phrases = Phrases(jobD, min_count=10, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[jobD]

In [ ]:
from gensim.models.phrases import Phrases, Phraser
cf_jobD = [row.split() for row in sf_model_final.loc[:, 'Skill description']]
cf_phrases = Phrases(cf_jobD, min_count=10, progress_per=10000)
cf_bigram = Phraser(cf_phrases)
cf_sentences = cf_bigram[cf_jobD]

In [ ]:
sf_model_final['Skill Description'] = sentences

In [ ]:
sf_model_final['Skill Description'] = sf_model_final['Skill Description'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
sf_model_final

In [ ]:
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job Role']

In [ ]:
sf_model_final = sf_model_final.reset_index().drop(columns = ['index'])
sf_model_final = sf_model_final.dropna()

In [ ]:
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job_Role_Replaced'].str.replace('Senior', '')
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job_Role_Replaced'].str.replace('Junior', '')
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job_Role_Replaced'].str.replace('Manager', '')
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job_Role_Replaced'].str.replace('Executive', '')
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job_Role_Replaced'].str.replace('Director', '')
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job_Role_Replaced'].str.replace('Lead', '')
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job_Role_Replaced'].str.replace('Head of ', '')
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job_Role_Replaced'].str.replace('(Specialist)', '')
sf_model_final['Job_Role_Replaced'] = sf_model_final['Job_Role_Replaced'].str.replace('Specialist', '')

In [ ]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import re
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

In [ ]:
def clear_hierarchy(title):
    
    patterns = ["Senior", "Junior", "Manager", "Executive", "Director", "Lead", "Head of", "(Specialist)", "Specialist"]
    for s in patterns:
        title = re.sub(s, '',title)
        
    split = title.split()
    if (split[0] == 'Assistant') or (split[0]=='Head'):
        title = " ".join(split[1:])

    if title[-2:]=="or":
        title = title[:-2].strip()+"ing"
   
    
    return title

In [ ]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def lemmatize_text(text):
    list_ = []
    for w in w_tokenizer.tokenize(text):
        list_.append(lemmatize_stemming(w))
    
    title = ' '.join(list_)
  
    return list_



sf_model_final['Job_Role_Normal'] = sf_model_final['Job Role'].apply(lambda x: clear_hierarchy(x))
sf_model_final['Job_Role_Stemmed'] = sf_model_final.Job_Role_Normal.apply(lemmatize_text)

In [ ]:
sf_model_final[sf_model_final['Sector']=="Infocomm Technology"]['Job_Role_Replaced'].unique()

In [ ]:
sf_model_final.to_csv("sf_model_final.csv")

In [ ]:
sf_model_final[(sf_model_final['Job Role']=='Marketing Manager') & (sf_model_final['Sector']=='Infocomm Technology')]['Skill Title'].unique()

In [ ]:
sf_model_final = pd.read_csv("sf_model_final.csv")

In [ ]:
sf_model_final

In [ ]:
sf_model_IT = sf_model_final[sf_model_final['Sector']=="Infocomm Technology"]

In [ ]:
cf_model_IT = cf_model[cf_model['Industry_unlisted']=="Information Technology"]

In [ ]:
X_train = sf_model_IT['Skill Description']
y_train = sf_model_IT['Job_Role_Replaced']

In [ ]:
X_test = cf_model_IT['Job Description']

In [ ]:
idx = sf_model_IT[sf_model_IT['Job Role'] == 'Associate Systems Support Engineer'].index

In [ ]:
X_train = X_train.drop(idx)
y_train = y_train.drop(idx)

In [ ]:

X_train_sf, X_test_sf, y_train_sf, y_test_sf = train_test_split(X_train, 
                                                    y_train, 
                                                    test_size = .3, 
                                                    shuffle = True, 
                                                    stratify = y_train, 
                                                    random_state = 3)

In [ ]:
# !pip install -U scikit-learn

In [ ]:
cf = pd.read_csv('CareersFutureCleaned.csv')

In [ ]:
#sf = pd.read_csv('sf_model_final_final.csv')
sf = pd.read_csv('sf_model_final.csv')

In [ ]:
sf['Skill Title'] = sf['Skill Title'].apply(lambda x: x.replace("['", "").replace("']", ""))

In [ ]:
sf

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                
               ])
logreg.fit(sf['Skill Title'], sf['Job Role'])



#y_pred = logreg.predict()



# print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred,target_names=ind))

In [ ]:
roles = sf['Job Role'].unique()

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import process, fuzz

In [ ]:
Str_Sort_Match = fuzz.partial_ratio('Science data', 'Data Scientist and DevSecOps Developer')
print("String Matched:",Str_Sort_Match)

In [ ]:
def check_similary_jobs(title):
    max = 50
    return_value = ""
    for role in roles:
        if fuzz.partial_ratio(title, role)>max:
            max = fuzz.partial_ratio(title, role)
            return_value = role
    return return_value
        
    

In [ ]:
cf_subset = cf[:2000]

In [ ]:
cf_subset['SFW_Job'] = cf_subset['Job Title'].apply(lambda x:check_similary_jobs(x) )

In [ ]:
test_data = cf_subset[cf_subset['SFW_Job']!='']

In [ ]:
test_data['Skills'] = test_data['Skills'].apply(lambda x: x.replace("[", "").replace("]", ""))

In [ ]:
y_pred = logreg.predict(test_data['Skills'])

In [ ]:
y_pred

In [ ]:
test_data['SFW_Job']

In [ ]:
accuracy_score(test_data['SFW_Job'], y_pred)

In [ ]:
test_data.to_csv("test_set.csv")

In [ ]:
test_data

In [ ]:
sf = pd.read_csv('sf_model_final_final.csv')
sf['Skill Title'] = sf['Skill Title'].apply(lambda x: x.replace("['", "").replace("']", ""))


In [ ]:
sf

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                
               ])
logreg.fit(sf['Skill Title'], sf['Job Role'])


In [ ]:
test_data = pd.read_csv('test_set.csv')

In [ ]:
test_data

In [ ]:
y_pred = logreg.predict(test_data['Skills'])

In [ ]:
y_pred[0]

In [ ]:
test_data['SFW_Job'][0]

In [ ]:
def acc(df, pred):
    counter = 0
    for i in range(len(df)):
        if df['SFW_Job'][i] in pred[i]:
            counter+=1
            print(df['SFW_Job'][i])
            print(pred[i])
    
    return counter/len(df)
            

In [ ]:
acc(test_data, y_pred)

In [ ]:
accuracy_score(y_test_sf, y_pred)

In [ ]:
results = pd.DataFrame(X_test).reset_index()
Y_test = pd.DataFrame(cf_model_IT['Job Title']).reset_index()
Y_pred = pd.DataFrame(y_pred).reset_index()
results = pd.concat([results, Y_test, Y_pred[0]], axis = 1, ignore_index=True)
results = results.drop(columns = [0,2])
results.columns = ["CF Decription", "CF Job Title", "SF Job Title Matching"]
results.to_csv("log_results_no_bigram.csv", index = False)

In [ ]:
# from gensim.models import Word2Vec

# wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
# wv.init_sims(replace=True)

In [ ]:
DT = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier(random_state=3)),
                
               ])
DT.fit(X_train_sf, y_train_sf)


y_pred = DT.predict(X_test_sf)

In [ ]:
accuracy_score(y_test_sf, y_pred)

In [ ]:
X_test

In [ ]:
y_pred

In [ ]:
# X_train = sf_model_IT['Skill description']

KN = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', KNeighborsClassifier()),
                
               ])
KN.fit(X_train_sf, y_train_sf)


y_pred = KN.predict(X_test_sf)

In [ ]:
accuracy_score(y_test_sf, y_pred)

In [ ]:
y_pred

In [ ]:
results = pd.DataFrame(X_test).reset_index()
Y_test = pd.DataFrame(cf_model_IT['Job Title']).reset_index()
Y_pred = pd.DataFrame(y_pred).reset_index()
results = pd.concat([results, Y_test, Y_pred[0]], axis = 1, ignore_index=True)
results = results.drop(columns = [0,2])
results.columns = ["CF Decription", "CF Job Title", "SF Job Title Matching"]
results.to_csv("KN_no_bigrams.csv", index = False)

In [ ]:

RF = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators = 50)),
                
               ])
RF.fit(X_train_sf, y_train_sf)


y_pred = RF.predict(X_test_sf)

In [ ]:
accuracy_score(y_test_sf, y_pred)

In [ ]:
SGD = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(alpha=1e-06,
               loss='log',
               max_iter=1000,
               penalty='l2',
               learning_rate = 'constant',
               eta0 = .1,
               random_state = 3,
               tol=None)),
                
               ])
SGD.fit(X_train, y_train)


y_pred = SGD.predict(X_test)


In [ ]:
y_pred

In [ ]:
results = pd.DataFrame(X_test).reset_index()
Y_test = pd.DataFrame(cf_model_IT['Job Title']).reset_index()
Y_pred = pd.DataFrame(y_pred).reset_index()
results = pd.concat([results, Y_test, Y_pred[0]], axis = 1, ignore_index=True)
results = results.drop(columns = [0,2])
results.columns = ["CF Decription", "CF Job Title", "SF Job Title Matching"]
results.to_csv("SGD_no_bigrams.csv", index = False)

In [ ]:
SGD = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(alpha=1e-06,
               loss='log',
               max_iter=1000,
               penalty='l2',
               learning_rate = 'constant',
               eta0 = .1,
               random_state = 3,
               tol=None)),
                
               ])
SGD.fit(X_train_sf, y_train_sf)


y_pred_sf = SGD.predict(X_test_sf)


In [ ]:
accuracy_score(y_test_sf, y_pred_sf)

In [ ]:
y_test_sf

In [ ]:
y_pred_sf

In [ ]:
cm = confusion_matrix(y_test_sf, y_pred_sf) 

cm_df = pd.DataFrame(cm,
                     index = y_train_sf.unique(), 
                     columns = y_train_sf.unique())

#Plot the heatmap
plt.figure(figsize=(20, 20))

sns.heatmap(cm_df, 
            center=0, 
            cmap=sns.diverging_palette(220, 15, as_cmap=True), 
            annot=True, 
            fmt='g')

plt.title('SGD (loss = log) \nF1 Score (avg = macro) : {0:.2f}'.format(f1_score(y_test_sf, y_pred_sf, average='macro')), fontsize = 13)
plt.ylabel('True label', fontsize = 13)
plt.xlabel('Predicted label', fontsize = 13)
plt.show()

In [ ]:
# sf_model_IT[sf_model_IT['Skill Title']=="Data Design"]
sf_model_IT[sf_model_IT['Job Role']=="Data Engineer"]

In [ ]:
# !pip3 install tensorflow --upgrade

In [ ]:
sf_jobs_concatenated = sf_model_IT.groupby(['Job Role','Skill Title'])['Skill description'].unique()
sf_jobs_concatenated = pd.DataFrame(sf_jobs_concatenated)

sf_jobs_concatenated = sf_jobs_concatenated.reset_index(level=[0,1])

sf_jobs_concatenated  = sf_jobs_concatenated[["Job Role", "Skill description"]].groupby(['Job Role'])
sf_jobs_concatenated  = pd.DataFrame(sf_jobs_concatenated  )

In [ ]:
sf_jobs_concatenated

In [ ]:
def concat_skills(df):
    jd_list = []

    for row in range(len(df)):
        jd=""
        for val in df.iloc[row][1]['Skill description']:
            jd+=val
            
        jd_list.append(jd)
    return jd_list

jd_list = concat_skills(sf_jobs_concatenated)



In [ ]:
sf_jobs_concatenated['Job description'] = jd_list
sf_jobs_concatenated = sf_jobs_concatenated.drop(columns = [1])
sf_jobs_concatenated.columns = ['Job_Role', 'Job description']
sf_jobs_concatenated

In [ ]:
sf_jobs_concatenated['Job description'] = [','.join(map(str, l)) for l in sf_jobs_concatenated['Job description']]


In [ ]:
sf_jobs_concatenated

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                
               ])
logreg.fit(sf_jobs_concatenated['Job description'], sf_jobs_concatenated['Job_Role'])


y_pred = logreg.predict(X_test)

In [ ]:
y_pred

In [ ]:
list(set(y_pred))

In [ ]:
text_list = [text_resume, text]
cv = CountVectorizer()
count_matrix = cv.fit_transform(text_list)

In [ ]:
sf_skills = pd.read_csv("sf_skills_concat.csv")

In [ ]:
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from gensim.models.phrases import Phrases, Phraser

In [ ]:

jobD = [row.split() for row in sf_skills['Skill description'].astype(str)]
phrases = Phrases(jobD, min_count=10, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[jobD]


In [ ]:
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=10,
                     window=3,
                     size=55,
                     sample=6e-5, 
                     alpha=0.05, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
w2v_model.build_vocab(sentences, progress_per=10000)

In [ ]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=40, report_delay=1)

In [ ]:
w2v_model.wv.most_similar(positive=["health"])

In [ ]:
w2v_model.wv.most_similar(positive=["finance"])

In [ ]:
w2v_model.wv.doesnt_match(['bio', 'medicine', 'computer'])

In [ ]:
vocabs = []
vecs = []
for word in w2v_model.wv.vocab:
    vocabs.append(word)
    vecs.append(w2v_model[word])

In [ ]:
sub_vecs = vecs[:100]
sub_vocab = vocabs[:100]
tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2000, random_state=23)
new_values = tsne_model.fit_transform(sub_vecs)

In [ ]:
%matplotlib inline

x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])
        
plt.figure(figsize=(16, 16)) 
for i in range(len(x)):
    plt.scatter(x[i],y[i])
    plt.annotate(sub_vocab[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
plt.show()

BERT

In [ ]:
# !pip install pytorch

In [ ]:
# !pip install torchtext

In [ ]:
sf_model_IT.columns = [c.replace(' ', '_') for c in sf_model_IT.columns]

In [ ]:
sf_model_IT_BERT = sf_model_IT.copy() 

In [ ]:
sf_model_IT_BERT

In [ ]:
possible_labels = sf_model_IT.Job_Role.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

In [ ]:
sf_model_IT_BERT['label'] = sf_model_IT_BERT.Job_Role.replace(label_dict)

In [ ]:
sf_model_IT_BERT['label'].value_counts()

In [ ]:
sf_model_IT_BERT = sf_model_IT_BERT.drop(sf_model_IT_BERT[sf_model_IT_BERT['label'] == 16].index)

In [ ]:
sf_model_IT_BERT

In [ ]:
sf_model_IT_BERT.to_csv("sf_model_IT_Bert.csv")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(sf_model_IT_BERT.index.values, 
                                                  sf_model_IT_BERT.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=sf_model_IT_BERT.label.values)



In [ ]:
sf_model_IT_BERT['data_type'] = ['not_set']*sf_model_IT_BERT.shape[0]

sf_model_IT_BERT.loc[X_train, 'data_type'] = 'train'
sf_model_IT_BERT.loc[X_val, 'data_type'] = 'val'



In [ ]:
sf_model_IT_BERT[['Job_Role', 'Skill_description', 'data_type', 'label']].groupby(['Job_Role', 'label', 'data_type']).count()

In [ ]:

#from tqdm.notebook import tqdm

#
#from torch.utils.data import TensorDataset

# from transformers import BertForSequenceClassification



In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    sf_model_IT_BERT[sf_model_IT_BERT.data_type=='train'].Skill_description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    sf_model_IT_BERT[sf_model_IT_BERT.data_type=='val'].Skill_description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(sf_model_IT_BERT[sf_model_IT_BERT.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(sf_model_IT_BERT[sf_model_IT_BERT.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:


seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [ ]:
device = torch.device("cuda")
import random

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

EDA

In [ ]:
top_20 = hero.visualization.top_words(sf_model_final.loc[:, 'Skill description']).head(20)
top_20.plot.bar(rot=90, title="Top 20 words in corpus");
plt.show(block=True)

In [ ]:
sf_model_final['Skill Title'].value_counts().sort_values(ascending=False).head(20).plot(kind='bar')

In [ ]:
hero.wordcloud(sf_model_final.loc[:, 'Skill description'], max_words=100)

In [ ]:
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=10,
                     window=3,
                     size=55,
                     sample=6e-5, 
                     alpha=0.05, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
w2v_model.build_vocab(sentences, progress_per=10000)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)
vocabs = []
vecs = []


In [ ]:
for word in w2v_model.wv.vocab:
    vocabs.append(word)
    vecs.append(w2v_model[word])
sub_vecs = vecs[:100]
sub_vocab = vocabs[:100]
tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2000, random_state=23)
new_values = tsne_model.fit_transform(sub_vecs)

In [ ]:
%matplotlib inline

x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])
        
plt.figure(figsize=(16, 16)) 
for i in range(len(x)):
    plt.scatter(x[i],y[i])
    plt.annotate(sub_vocab[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
plt.show()

In [ ]:
cores = multiprocessing.cpu_count()
w2v_model_cf = Word2Vec(min_count=10,
                     window=3,
                     size=55,
                     sample=6e-5, 
                     alpha=0.05, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
w2v_model_cf.build_vocab(cf_sentences, progress_per=10000)
w2v_model_cf.train(cf_sentences, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)
vocabs = []
vecs = []

In [ ]:
for word in w2v_model_cf.wv.vocab:
    vocabs.append(word)
    vecs.append(w2v_model_cf[word])
sub_vecs = vecs[:100]
sub_vocab = vocabs[:100]
tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2000, random_state=23)
new_values = tsne_model.fit_transform(sub_vecs)

In [ ]:
%matplotlib inline

x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])
        
plt.figure(figsize=(16, 16)) 
for i in range(len(x)):
    plt.scatter(x[i],y[i])
    plt.annotate(sub_vocab[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
plt.show()

Unnecessary stuff lol

In [ ]:
!pip install autocorrect
from autocorrect import Speller

spell = Speller(lang='en')

print (spell('hillo'))

In [ ]:
print(spell('organization'))

In [ ]:

!pip install pyspellchecker
from spellchecker import SpellChecker
spell = SpellChecker(distance=1)
test = ['na', 'lief']
for word in test:
  # Get the one `most likely` answer
  print(spell.correction(word))

In [ ]:
!pip install tqdm

In [ ]:
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources
import re, string, json
import spacy
from tqdm import tqdm

In [ ]:
#!pip install symspellpy
from symspellpy import SymSpell
from itertools import islice
import pkg_resources
def spell_correction(sentence_list):
    max_edit_distance_dictionary= 3
    prefix_length = 4
    spellchecker = SymSpell(max_edit_distance_dictionary, prefix_length)
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt")
    bigram_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
    spellchecker.load_dictionary(dictionary_path, term_index=0, count_index=1)
    spellchecker.load_bigram_dictionary(dictionary_path, term_index=0, count_index=2)
    norm_sents = []
    print("Spell correcting")
    for sentence in tqdm(sentence_list):
        norm_sents.append(_spell_correction_text(sentence, spellchecker))
    return norm_sents

def _spell_correction_text(text, spellchecker):
    """
    This function does very simple spell correction normalization using pyspellchecker module. It works over a tokenized sentence and only the token representations are changed.
    """
    if len(text) < 1:
        return ""
    #Spell checker config
    max_edit_distance_lookup = 2
    suggestion_verbosity = Verbosity.TOP # TOP, CLOSEST, ALL
    #End of Spell checker config
    token_list = text.split()
    for word_pos in range(len(token_list)):
        word = token_list[word_pos]
        if word is None:
            token_list[word_pos] = ""
            continue
        if not '\n' in word and word not in string.punctuation and not is_numeric(word) and not (word.lower() in spellchecker.words.keys()):
            suggestions = spellchecker.lookup(word.lower(), suggestion_verbosity, max_edit_distance_lookup)
            #Checks first uppercase to conserve the case.
            upperfirst = word[0].isupper()
            
            if len(suggestions) > 0:
                correction = suggestions[0].term
                replacement = correction
            #We call our _reduce_exaggerations function if no suggestion is found. Maybe there are repeated chars.
            else:
                replacement = _reduce_exaggerations(word)

            #Takes the case back to the word.
            if upperfirst:
                replacement = replacement[0].upper()+replacement[1:]
            word = replacement
            token_list[word_pos] = word
    return " ".join(token_list).strip()

def _reduce_exaggerations(text):
    """
    Auxiliary function to help with exxagerated words.
    Examples:
        woooooords -> words
        yaaaaaaaaaaaaaaay -> yay
    """
    correction = str(text)
    #TODO work on complexity reduction.
    return re.sub(r'([\w])\1+', r'\1', correction)

def is_numeric(text):
    for char in text:
        if not (char in "0123456789" or char in ",%.$"):
            return False
    return True

In [ ]:
spell_correction("Hi, hellllo. How are you doin")

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd
# import torch

# # Preliminaries

# from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# # Models

# import torch.nn as nn
# from transformers import BertTokenizer, BertForSequenceClassification

# # Training

# import torch.optim as optim

# # Evaluation

# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import seaborn as sns

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Model parameter
# MAX_SEQ_LEN = 128
# PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
# UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)



# # Fields

# label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
# text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
#                    fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
# fields = [('label', label_field), ('title', text_field), ('text', text_field), ('titletext', text_field)]

# # TabularDataset

# train, valid, test = TabularDataset.splits(path=source_folder, train='train.csv', validation='valid.csv',
#                                            test='test.csv', format='CSV', fields=fields, skip_header=True)

# # Iterators

# train_iter = BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.text),
#                             device=device, train=True, sort=True, sort_within_batch=True)
# valid_iter = BucketIterator(valid, batch_size=16, sort_key=lambda x: len(x.text),
#                             device=device, train=True, sort=True, sort_within_batch=True)
# test_iter = Iterator(test, batch_size=16, device=device, train=False, shuffle=False, sort=False)

# class BERT(nn.Module):

#     def __init__(self):
#         super(BERT, self).__init__()

#         options_name = "bert-base-uncased"
#         self.encoder = BertForSequenceClassification.from_pretrained(options_name)

#     def forward(self, text, label):
#         loss, text_fea = self.encoder(text, labels=label)[:2]

#         return loss, text_fea

# def save_checkpoint(save_path, model, valid_loss):

#     if save_path == None:
#         return
    
#     state_dict = {'model_state_dict': model.state_dict(),
#                   'valid_loss': valid_loss}
    
#     torch.save(state_dict, save_path)
#     print(f'Model saved to ==> {save_path}')

# def load_checkpoint(load_path, model):
    
#     if load_path==None:
#         return
    
#     state_dict = torch.load(load_path, map_location=device)
#     print(f'Model loaded from <== {load_path}')
    
#     model.load_state_dict(state_dict['model_state_dict'])
#     return state_dict['valid_loss']


# def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

#     if save_path == None:
#         return
    
#     state_dict = {'train_loss_list': train_loss_list,
#                   'valid_loss_list': valid_loss_list,
#                   'global_steps_list': global_steps_list}
    
#     torch.save(state_dict, save_path)
#     print(f'Model saved to ==> {save_path}')


# def load_metrics(load_path):

#     if load_path==None:
#         return
    
#     state_dict = torch.load(load_path, map_location=device)
#     print(f'Model loaded from <== {load_path}')
    
#     return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

# def train(model,
#           optimizer,
#           criterion = nn.BCELoss(),
#           train_loader = train_iter,
#           valid_loader = valid_iter,
#           num_epochs = 5,
#           eval_every = len(train_iter) // 2,
#           file_path = destination_folder,
#           best_valid_loss = float("Inf")):
    
#     # initialize running values
#     running_loss = 0.0
#     valid_running_loss = 0.0
#     global_step = 0
#     train_loss_list = []
#     valid_loss_list = []
#     global_steps_list = []

#     # training loop
#     model.train()
#     for epoch in range(num_epochs):
#         for (labels, title, text, titletext), _ in train_loader:
#             labels = labels.type(torch.LongTensor)           
#             labels = labels.to(device)
#             titletext = titletext.type(torch.LongTensor)  
#             titletext = titletext.to(device)
#             output = model(titletext, labels)
#             loss, _ = output

#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             # update running values
#             running_loss += loss.item()
#             global_step += 1

#             # evaluation step
#             if global_step % eval_every == 0:
#                 model.eval()
#                 with torch.no_grad():                    

#                     # validation loop
#                     for (labels, title, text, titletext), _ in valid_loader:
#                         labels = labels.type(torch.LongTensor)           
#                         labels = labels.to(device)
#                         titletext = titletext.type(torch.LongTensor)  
#                         titletext = titletext.to(device)
#                         output = model(titletext, labels)
#                         loss, _ = output
                        
#                         valid_running_loss += loss.item()

#                 # evaluation
#                 average_train_loss = running_loss / eval_every
#                 average_valid_loss = valid_running_loss / len(valid_loader)
#                 train_loss_list.append(average_train_loss)
#                 valid_loss_list.append(average_valid_loss)
#                 global_steps_list.append(global_step)

#                 # resetting running values
#                 running_loss = 0.0                
#                 valid_running_loss = 0.0
#                 model.train()

#                 # print progress
#                 print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
#                       .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
#                               average_train_loss, average_valid_loss))
                
#                 # checkpoint
#                 if best_valid_loss > average_valid_loss:
#                     best_valid_loss = average_valid_loss
#                     save_checkpoint(file_path + '/' + 'model.pt', model, best_valid_loss)
#                     save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
#     save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
#     print('Finished Training!')

# model = BERT().to(device)
# optimizer = optim.Adam(model.parameters(), lr=2e-5)

# train(model=model, optimizer=optimizer)

# train_loss_list, valid_loss_list, global_steps_list = load_metrics(destination_folder + '/metrics.pt')
# plt.plot(global_steps_list, train_loss_list, label='Train')
# plt.plot(global_steps_list, valid_loss_list, label='Valid')
# plt.xlabel('Global Steps')
# plt.ylabel('Loss')
# plt.legend()
# plt.show() 

# def evaluate(model, test_loader):
#     y_pred = []
#     y_true = []

#     model.eval()
#     with torch.no_grad():
#         for (labels, title, text, titletext), _ in test_loader:

#                 labels = labels.type(torch.LongTensor)           
#                 labels = labels.to(device)
#                 titletext = titletext.type(torch.LongTensor)  
#                 titletext = titletext.to(device)
#                 output = model(titletext, labels)

#                 _, output = output
#                 y_pred.extend(torch.argmax(output, 1).tolist())
#                 y_true.extend(labels.tolist())
    
#     print('Classification Report:')
#     print(classification_report(y_true, y_pred, labels=[1,0], digits=4))
    
#     cm = confusion_matrix(y_true, y_pred, labels=[1,0])
#     ax= plt.subplot()
#     sns.heatmap(cm, annot=True, ax = ax, cmap='Blues', fmt="d")

#     ax.set_title('Confusion Matrix')

#     ax.set_xlabel('Predicted Labels')
#     ax.set_ylabel('True Labels')

#     ax.xaxis.set_ticklabels(['FAKE', 'REAL'])
#     ax.yaxis.set_ticklabels(['FAKE', 'REAL'])
    
# best_model = BERT().to(device)

# load_checkpoint(destination_folder + '/model.pt', best_model)

# evaluate(best_model, test_iter)